In [1]:
import numpy as np, pandas as pd, time, re
from datetime import datetime
from cmfrec import CMF
import os
from datetime import datetime
from copy import deepcopy

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [3]:
from src.data_pipeline.DataLoader import DataLoader
from src.utility.sys_utils import get_spark

# Read data from files

In [16]:
# change the config file to your own file, otherwise it will not find proper path
dataloader = DataLoader(dataset_name="user_10_item_1_exp", config_name="default_config.json")
spark = get_spark(cores=4)

# Display side info schema

In [17]:
user_side_info = dataloader.get_user_side_info()

In [18]:
user_side_info.head()

,average_stars,cool,fans,funny,review_count,useful,user_id,cnt_compliment,yelp_days,elite_years
0,4.03,25,5,17,95,84,l6BmjZMeQD3rDxWUbiAiow,8,2275,3
1,3.63,16,4,22,33,48,4XChL029mKr5hydo79Ljxg,3,2504,0
2,3.71,10,0,8,16,28,bc8C_eETBWL0olvFSJJd0w,1,2279,0
3,4.85,14,5,4,17,30,dD0gZpBctWGdWo9WlGuhlA,4,2049,0
4,4.08,665,39,279,361,1114,MM4RJAeH6yuaN8oZDSt0RA,293,2260,4


In [19]:
item_side_info = dataloader.get_item_side_info()

In [20]:
item_side_info.head()

,business_id,top_category,review_count,stars,state,city
0,1SWheh84yJXfytovILXOAQ,Active Life,5,3.0,AZ,Phoenix
1,QXAEGFB4oINsVuTFxEYKFQ,Food,128,2.5,ON,Mississauga
2,gnKjwL_1w79qoiV3IC_xQQ,Restaurants,170,4.0,NC,Charlotte
3,xvX2CttrVhyG2z1dFg_0xw,Financial Services,3,5.0,AZ,Goodyear
4,HhyxOkGAM07SRYtlQ4wMFQ,Shopping,4,4.0,NC,Charlotte


# Fit with item side information

In [21]:
from src.model.CollectiveMF import CollectiveMF

In [22]:
param_dict = {
    "w_main": 1.0,
    "use_user_info": False,
    "use_item_info": True,
    "k": 5,
    "reg_param": 1e-4
}

In [23]:
cmf = CollectiveMF(params=param_dict)

In [24]:
from src.data_pipeline.pipeline import test_evaluation

In [25]:
test_evaluation(data_loader=dataloader, model=cmf, spark=spark, caching=True)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 6.857606
  Number of iterations: 194
  Number of functions evaluations: 233
Creating directory and start writing ...
Rewriting files in /media/ExtHDD01/log/user_10_item_1_exp/CollectiveMF/k_5-reg_param_0.0001-w_main_1.0/Rating_fold_-1.parquet
Finish inference phase


ChainMap({'ndcg@10': 0.12318043328933682, 'precision@10': 0.02305574433234009}, {'rmse': 1.4246476420534175})